In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy
import os
import csv
from sklearn.metrics.pairwise import cosine_similarity

## Data Preprocessing

In [3]:
data_dir = r"./data"
files = os.listdir(data_dir)
print("Files in the folder:")
for file in files:
    print(file)

Files in the folder:
all_video_label.csv


In [109]:
# dfs = []
# for file in files:
#     if (file == "all_video_label.csv"): continue
#     file_path = os.path.join(data_dir, file)
#     df = pd.read_csv(file_path)
#     df.fillna(0, inplace=True)
#     df['video_id'] = df['video_id'].str.replace('-', '_')
#     temp_df = df.drop(columns=['video_id'])
#     df = df[(temp_df != 0).any(axis=1)]
#     dfs.append(df)
# print(dfs[4].head()) 
# data = None
# for df in dfs:
#     data = pd.concat([data, df], axis=0)
# data = data.reset_index(drop=True)
# print(len(data))
# data.to_csv("./data/all_video_label.csv", index=True, encoding="utf-8")
# data = data.drop(columns=['video_id'])

212


# Load data

In [6]:
data = pd.read_csv(os.path.join(data_dir, 'all_video_label.csv'))

## Cosine Similarity test

In [390]:
user_watched_indices = [2, 100]
user_watched_data = data.iloc[user_watched_indices]

# Compute cosine similarity
cosine_similarities = cosine_similarity(user_watched_data, data)

# Get indices of videos sorted by similarity (excluding the user's watched videos)
video_indices = sorted(range(len(cosine_similarities[0])), key=lambda i: cosine_similarities[0][i], reverse=True)
recommended_indices = [index for index in video_indices if index not in user_watched_indices]

# Recommend the top N videos
top_n_recommendations = recommended_indices[:10]  # Replace N with the desired number of recommendations

# Display or use the recommended indices
print("Recommended Video Indices:", top_n_recommendations)

Recommended Video Indices: [37, 185, 211, 17, 19, 116, 132, 31, 25, 0]


## print the tag of given video_id

In [190]:
def search_tag(id):
    target = data.iloc[id]
    target = target[target != 0]
    print("the tag of id = " + str(id))
    for tag in target.index:
        print(tag)


## Cosine Similarity Recommendation

N : int    return N most similar video_id
user_watched_indices : list   the history watched list of user
watched_proportions : 

In [8]:
def CS(N, user_watched_indices, watched_proportions):
    """
    N: Nth recommended videos
    user_watched_incides: User watched video list
    watched_proportions: Watched video (proportional) time
    """
    user_watched_data = data.iloc[user_watched_indices]
    user_watched_data = user_watched_data.reset_index(drop=True)
    
    sum = user_watched_data.sum()
    print("======= the avg vector =======")
    print(sum[sum != 0])
    
    # add up a noise on the watching_proportion   
    # watching_propotions += np.random.uniform(0.2, 0.5, len(watching_propotions))

    # weigthed the data with watching proportion
    for ind, row in user_watched_data.iterrows():
        row = row * watched_proportions[ind]
        user_watched_data.loc[ind] = row

    sum = user_watched_data.sum()
    print("======= the weighted vector =======")
    print(sum[sum != 0])

    # Compute cosine similarity
    cosine_similarities = cosine_similarity([sum], data)

    # Get indices of videos sorted by similarity (excluding the user's watched videos)
    video_indices = sorted(range(len(cosine_similarities[0])), key=lambda i: cosine_similarities[0][i], reverse=True)
    recommended_indices = [index for index in video_indices if index not in user_watched_indices]

    # Recommend the top N videos
    top_n_recommendations = recommended_indices[:N]  # Replace N with the desired number of recommendations

    # Display or use the recommended indices
    print("\n============")
    print("Recommended Video Indices:", top_n_recommendations)
    for rec in top_n_recommendations:
        search_tag(rec)
    return top_n_recommendations

In [373]:
CS(10, [2, 4], [1, 1])


======= the avg vector =======
多項式的運算與應用    1.0
直角三角形邊角關係    1.0
dtype: float64
======= the weighted vector =======
多項式的運算與應用    1.0
直角三角形邊角關係    1.0
dtype: float64

Recommended Video Indices: [25, 37, 58, 68, 166, 185, 186, 198, 211, 17]
the tag of id = 25
多項式的運算與應用
直角三角形邊角關係
正弦定理、餘弦定理
三角測量
the tag of id = 37
直角三角形邊角關係
the tag of id = 58
多項式的運算與應用
the tag of id = 68
多項式的運算與應用
the tag of id = 166
多項式的運算與應用
the tag of id = 185
直角三角形邊角關係
the tag of id = 186
多項式的運算與應用
the tag of id = 198
多項式的運算與應用
the tag of id = 211
直角三角形邊角關係
the tag of id = 17
直角三角形邊角關係
三角測量


In [388]:
import random

random_watched_indicies_array = np.random.randint(1, 200, 10)
random_watched_proportion_array = np.random.uniform(0, 1, 10)
print(random_watched_indicies_array)
print(random_watched_proportion_array)
CS(10, random_watched_indicies_array, random_watched_proportion_array)


[150 194 142  15 102  75 178   3 170 170]
[0.44039476 0.15150458 0.57586236 0.24727532 0.32121872 0.86498591
 0.80998783 0.8663161  0.957648   0.80277531]
======= the avg vector =======
線性規劃              2.0
正弦定理、餘弦定理         1.0
指數                2.0
常用對數              2.0
平面向量的內積           4.0
矩陣的運算             1.0
矩陣的應用             1.0
條件機率、貝氏定理、獨立事件    2.0
dtype: float64
======= the weighted vector =======
線性規劃              1.760423
正弦定理、餘弦定理         0.321219
指數                0.398780
常用對數              0.398780
平面向量的內積           3.491725
矩陣的運算             0.440395
矩陣的應用             0.440395
條件機率、貝氏定理、獨立事件    1.385850
dtype: float64

Recommended Video Indices: [6, 9, 27, 51, 189, 200, 35, 47, 93, 146]
the tag of id = 6
平面向量的內積
the tag of id = 9
平面向量的內積
the tag of id = 27
平面向量的內積
the tag of id = 51
平面向量的內積
the tag of id = 189
平面向量的內積
the tag of id = 200
平面向量的內積
the tag of id = 35
正弦定理、餘弦定理
平面向量的內積
the tag of id = 47
正弦定理、餘弦定理
平面向量的內積
the tag of id = 93
正弦定理、餘弦定理
平面向量的內積
the tag of id

## Collaborative Filtering User Based

### create random users

In [356]:
user1 = {
    'watched_indicies' : [1, 10 , 4, 47, 60, 88],
    'watched_proportion' : [0.5, 0.3, 0.2, 1, 0.8, 0.2]
}
user2 = {
    'watched_indicies' : [1, 10 ],
    'watched_proportion' : [0.5, 1]
}

user3 = {
    'watched_indicies' : [23, 45, 60, 12, 33],
    'watched_proportion' : [0.5, 1, 1]
}

random_user_count = 10
users = []

for i in range(random_user_count):
    random_watched_indicies_array = np.random.randint(1, 200, 10)
    random_watched_proportion_array = np.random.uniform(0, 1, 10)
    users.append({
         'watched_indicies' : random_watched_indicies_array,
         'watched_proportion' : random_watched_proportion_array
    })


In [355]:
def find_similar_user(N, quried_user_id, users):
    # weigthed the data with watching proportion
    quried_user = users[quried_user_id]
    user_watched_data = data.iloc[quried_user['watched_indicies']]
    quried_vec = user_watched_data.reset_index(drop=True)
    for ind, row in quried_vec.iterrows():
        row = row * quried_user['watched_proportion'][ind]
        quried_vec.loc[ind] = row
    quried_vec = quried_vec.sum().to_frame().T


    users_vecs = None

    for user in users:
        user_watched_data = data.iloc[user['watched_indicies']]
        feature_vec = user_watched_data.reset_index(drop=True)
        for ind, row in feature_vec.iterrows():
            row = row * user['watched_proportion'][ind]
            feature_vec.loc[ind] = row
        feature_vec = feature_vec.sum().to_frame().T
        users_vecs = pd.concat([users_vecs, feature_vec], axis=0)
    
    # Compute cosine similarity
    cosine_similarities = cosine_similarity(quried_vec, users_vecs)

    # Get indices of users sorted by similarity
    user_indices = sorted(range(len(cosine_similarities[0])), key=lambda i: cosine_similarities[0][i], reverse=True)
    similar_user_indicies = [index for index in user_indices if index != quried_user_id]

    # Recommend the top N videos
    top_n_similar_users_indices = similar_user_indicies[:N] 
    # Replace N with the desired number of recommendations
    print(top_n_similar_users_indices)

In [358]:
find_similar_user(6, 0, users)

[3, 4, 6, 2, 8, 1]


In [384]:
def find_similar_video_from_users(N, quried_user_id, similar_user_ids, users):
    video_indices_from_users = []

    for user_id in similar_user_ids:
        video_indices_from_users = np.concatenate([video_indices_from_users, users[user_id]['watched_indicies']])
    
    video_indices_from_users = np.unique(video_indices_from_users).astype(int)

    data_from_users = data.iloc[video_indices_from_users]
    #data_from_users.reset_index(drop=True)

    quried_user = users[quried_user_id]
    user_watched_data = data.iloc[quried_user['watched_indicies']]
    quried_vec = user_watched_data.reset_index(drop=True)
    for ind, row in quried_vec.iterrows():
        row = row * quried_user['watched_proportion'][ind]
        quried_vec.loc[ind] = row
    quried_vec = quried_vec.sum().to_frame().T

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(quried_vec, data_from_users)

    # Get indices of videos sorted by similarity (excluding the user's watched videos)
    indices = sorted(range(len(cosine_similarities[0])), key=lambda i: cosine_similarities[0][i], reverse=True)
    video_indices = video_indices_from_users[indices]
    recommended_indices = [index for index in video_indices if index not in users[quried_user_id]["watched_indicies"]]

    # Recommend the top N videos
    top_n_recommendations = recommended_indices[:N]  # Replace N with the desired number of recommendations
    print(top_n_recommendations)

    return top_n_recommendations
    
    

In [386]:
def feature_vec_user(user_id, users):
    quried_user = users[user_id]
    user_watched_data = data.iloc[quried_user['watched_indicies']]
    quried_vec = user_watched_data.reset_index(drop=True)
    for ind, row in quried_vec.iterrows():
        row = row * quried_user['watched_proportion'][ind]
        quried_vec.loc[ind] = row
    quried_vec = quried_vec.sum().to_frame().T
    result = quried_vec.loc[0]
    print(result[result != 0].sort_values(ascending=False)[:3])

In [387]:
similar_users_id = [1, 2]
quried_user_id = 0
N_similar_videos = 8

recommended_indices = find_similar_video_from_users(N_similar_videos, quried_user_id, similar_users_id, users)
for ind in recommended_indices:
    search_tag(ind)

for id in similar_users_id:
    feature_vec_user(id, users)

[55, 47, 146, 13, 6, 116, 151, 126]
the tag of id = 55
條件機率、貝氏定理、獨立事件
the tag of id = 47
正弦定理、餘弦定理
平面向量的內積
the tag of id = 146
正弦定理、餘弦定理
平面向量的內積
the tag of id = 13
平面方程式
平面上的線性變化與二階方陣
the tag of id = 6
平面向量的內積
the tag of id = 116
直角三角形邊角關係
正弦定理、餘弦定理
the tag of id = 151
圓與直線關係
平面向量的內積
the tag of id = 126
正弦定理、餘弦定理
和角公式與差角公式
平面向量的內積      2.970609
正弦定理、餘弦定理    2.120538
多項式的運算與應用    1.074515
Name: 0, dtype: float64
和角公式與差角公式    1.097797
正弦定理、餘弦定理    0.983727
三角測量         0.883289
Name: 0, dtype: float64


In [6]:
import pandas as pd
data = {
    "第一冊": [0, 1, 0, 1, 0],
    "第二冊": [1, 0, 1, 0, 0],
    "Value": [10, 20, 30, 40, 50]
}

df = pd.DataFrame(data)

# Use boolean indexing to filter rows
filtered_df = df[(df["第一冊"] == 1) | (df["第二冊"] == 1)]

# Print the resulting DataFrame
print(filtered_df.sample(2))

   第一冊  第二冊  Value
0    0    1     10
1    1    0     20


In [2]:
data = pd.read_csv("./data/all_video_label_v3.csv")
data[1:10]

,video_id,數與數線,式的運算,直線方程式與其圖形,線性規劃,圓與直線關係,多項式的運算與應用,簡單多項式函數及其圖形,多項式函數的徒刑與多項式不等式,數列,...,矩陣的運算,矩陣的應用,平面上的線性變化與二階方陣,樣本空間與事件,機率的定義與性質,條件機率、貝氏定理、獨立事件,第一冊,第二冊,第三冊,第四冊
1,110-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,110-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,110-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,110-5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,110-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,110-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,110-9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
8,110-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,110-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
for ind, row in data.iterrows():
    if (len(row[0].split("-")[-1]) < 2):
        print(row[0])
        row[0] = row[0].split("-")[0] + "-0" + row[0].split("-")[-1]
        data.iloc[ind] = row
        print(row[0])

data = data.sort_values(by=['video_id'])
data.reset_index(drop=True)
print(data[1:40])
data.to_csv("./data/all_video_label_v4.csv", index=False, encoding="utf-8")



    video_id  數與數線  式的運算  直線方程式與其圖形  線性規劃  圓與直線關係  多項式的運算與應用  簡單多項式函數及其圖形  \
155   102-09   0.0   0.0        0.0   0.0     0.0        0.0          1.0   
156   102-10   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
157   102-12   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
158   102-13   0.0   1.0        0.0   0.0     0.0        0.0          0.0   
159   103-01   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
160   103-02   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
161   103-03   0.0   0.0        0.0   0.0     1.0        0.0          0.0   
162   103-04   0.0   1.0        0.0   0.0     0.0        0.0          0.0   
163   103-05   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
164   103-06   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
165   103-07   0.0   0.0        0.0   0.0     0.0        0.0          1.0   
166   103-08   0.0   1.0        0.0   0.0     0.0        0.0          0.0   

/var/folders/39/3yfdyv352lg2hlp0c91r89yh0000gn/T/ipykernel_46570/3265938414.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (len(row[0].split("-")[-1]) < 2):


In [27]:
data = pd.read_csv("./data/all_video_label_v4.csv")

for ind, row in data.iterrows():
    if (row[0].split("-")[-1][0] == "0"):
        #print(row[0])
        row[0] = row[0].split("-")[0] + "-" + row[0].split("-")[-1][1]
        data.iloc[ind] = row
        
print(data[1:40])
data.to_csv("./data/all_video_label_v4.csv", index=False, encoding="utf-8")

/var/folders/39/3yfdyv352lg2hlp0c91r89yh0000gn/T/ipykernel_46570/3998442485.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[0].split("-")[-1][0] == "0"):
/var/folders/39/3yfdyv352lg2hlp0c91r89yh0000gn/T/ipykernel_46570/3998442485.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])
/var/folders/39/3yfdyv352lg2hlp0c91r89yh0000gn/T/ipykernel_46570/3998442485.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

102-08
102-09
103-01
103-02
103-03
103-04
103-05
103-06
103-07
103-08
103-09
104-01
104-02
104-03
104-04
104-05
104-06
104-07
104-09
105-01
105-02
105-03
105-04
105-05
105-06
105-07
105-08
105-09
106-01
106-02
106-04
106-05
106-06
106-07
106-08
106-09
107-01
107-02
107-03
107-04
107-05
107-06
107-07
107-08
107-09
108-01
108-02
108-03
108-04
108-05
108-06
108-07
108-08
108-09
109-01
109-02
109-03
109-04
109-05
109-06
109-07
109-08
109-09
110-01
110-02
110-03
110-04
110-05
110-07
110-08
110-09
111-01
111-02
111-03
111-04
111-05
111-06
111-07
111-08
111-09
112-01
112-02
112-03
112-04
112-05
112-06
112-07
112-08
112-09
   video_id  數與數線  式的運算  直線方程式與其圖形  線性規劃  圓與直線關係  多項式的運算與應用  簡單多項式函數及其圖形  \
1     102-9   0.0   0.0        0.0   0.0     0.0        0.0          1.0   
2    102-10   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
3    102-12   0.0   0.0        0.0   0.0     0.0        0.0          0.0   
4    102-13   0.0   1.0        0.0   0.0     0.0        0.0          0.0 